# 工具和路由 Tools and Routing

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、通过Langchain定义工具](#二、通过Langchain定义工具)
     - [2.1 通过装饰器之间定义Tool](#2.1-通过装饰器之间定义Tool)
     - [2.2 通过pydantic类定义Tool](#2.2-通过pydantic类定义Tool)
     - [2.3 天气查询应用案例](#2.3-天气查询应用案例)
     - [2.4 通过tool定义function](#2.4-通过tool定义function)
     - [2.5 通过API定义function案例](#2.5-通过API定义function案例)
 - [三、 路由](#三、路由)
     - [3.1 Tool转换为Function](#3.1-Tool转换为Function)
     - [3.2 通过route进行tools的选择](#3.2-通过route进行tools的选择)
     - [3.3 输出解析器](#3.3-输出解析器)
 - [四、总结](#四、总结)


# 一、设置OpenAI-API-Key

详细内容见`设置OpenAI_API_KEY.ipynb`文件

# 二、通过Langchain定义工具

## 2.1-通过装饰器之间定义Tool

In [1]:
from langchain.agents import tool
# tool装饰器包装了search函数
@tool
def search(query: str) -> str:
    """在网络上查询天气"""
    return "42度"

In [2]:
# 搜索工具的函数名
print(search.name)
#搜索工具的功能描述（即函数注释）
print(search.description)
# 搜索工具需要传递的参数
print(search.args)

search
search(query: str) -> str - 在网络上查询天气
{'query': {'title': 'Query', 'type': 'string'}}


## 2.2-通过pydantic类定义Tool

In [3]:
# 导入 Pydantic 库中的 BaseModel 类和 Field 函数，它们用于定义数据模型和字段。
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    """
    定义了 SearchInput 类中的一个属性 query，它是一个字符串类型。通过 Field 函数，你为这个字段提供了一些配置
    其中 description 参数用于描述这个字段的用途，即 "Thing to search for"（要搜索的内容）。
    """
    query: str = Field(description="你需要搜索的东西")

In [4]:
# 搜索工具类需要传递的参数
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [5]:
# 如果出现报错：ValidationError: 1 validation error for StructuredTool；需要安装特定版本的pydantic：pip install pydantic==1.10.8
# args_schema参数传递SearchInput工具类
@tool(args_schema=SearchInput)
def search_zh(query: str) -> str:
    """在网上查找温度"""
    return "42度"

In [6]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [7]:
search.run("圣弗朗西斯科")

'42度'

## 2.3-天气查询应用案例
整体代码逻辑：
1. 使用 Pydantic 定义了输入类OpenMeteoInput，以及输入的两个参数（经度和纬度）的输入格式
2. 定义了一个函数 get_current_temperature，该函数使用 OpenMeteo API 获取给定坐标位置的当前温度。
3. get_current_temperature函数通过发送 HTTP 请求获取 API 响应，然后从响应中提取并计算出当前时间对应的温度。
4. get_current_temperature函数返回一个字符串，其中包含了当前温度的信息。

In [8]:
# 导入所需的库
import requests
from pydantic import BaseModel, Field
import datetime

# 定义输入类（input schema）
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="要获取天气数据的位置的纬度") 
    longitude: float = Field(..., description="要获取天气数据的位置的经度") 

# 使用 @tool 装饰器并指定输入模型
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """"获取给定坐标的温度"""
    
    # Open Meteo API 的URL
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # 请求参数
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # 发送 API 请求
    response = requests.get(BASE_URL, params=params)
    
    # 检查响应状态码
    if response.status_code == 200:
        # 解析 JSON 响应
        results = response.json()
    else:
        # 处理请求失败的情况
        raise Exception(f"API Request failed with status code: {response.status_code}")

    # 获取当前 UTC 时间
    current_utc_time = datetime.datetime.utcnow()
    
    # 将时间字符串转换为 datetime 对象
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    
    # 获取温度列表
    temperature_list = results['hourly']['temperature_2m']
    
    # 找到最接近当前时间的索引
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    
    # 获取当前温度
    current_temperature = temperature_list[closest_time_index]
    
    # 返回当前温度的字符串形式
    return f'当前的温度是 {current_temperature}°C'


In [9]:
# 工具的名字
print(get_current_temperature.name)
# 工具的功能描述
print(get_current_temperature.description)
# 工具的输入参数
print(get_current_temperature.args)

get_current_temperature
get_current_temperature(latitude: float, longitude: float) -> dict - "获取给定坐标的温度
{'latitude': {'title': 'Latitude', 'description': '要获取天气数据的位置的纬度', 'type': 'number'}, 'longitude': {'title': 'Longitude', 'description': '要获取天气数据的位置的经度', 'type': 'number'}}


## 2.4-通过tool定义function

In [10]:
# 导入openai的模板
from langchain.tools.render import format_tool_to_openai_function

In [11]:
# 将定义好的工具直接传入模板，打印tool的名字、描述和输入参数格式
format_tool_to_openai_function(get_current_temperature)

/Users/jingllinzhou/anaconda3/envs/extractor/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - "获取给定坐标的温度',
 'parameters': {'type': 'object',
  'properties': {'latitude': {'description': '要获取天气数据的位置的纬度',
    'type': 'number'},
   'longitude': {'description': '要获取天气数据的位置的经度', 'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [12]:
# 调用工具
get_current_temperature({"latitude": 13, "longitude": 14})

/Users/jingllinzhou/anaconda3/envs/extractor/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/var/folders/6f/wm_jrpx102704v2s3pdk8lzm0000gn/T/ipykernel_12575/2418200860.py:39: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'当前的温度是 33.8°C'

In [13]:
import wikipedia

# 定义维基百科搜索的tool
@tool
def search_wikipedia(query: str) -> str:
    """打开维基百科搜索并获得页面的摘要"""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]: #取前三个页面标题
        try:
            #使用 wikipedia 模块的 page 函数，获取指定标题的维基百科页面对象。
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False) 
            # 获取页面摘要
            summaries.append(f"页面: {page_title}\n摘要: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "维基百科没有搜索到合适的结果"
    return "\n\n".join(summaries)

In [14]:
# 工具的名字
search_wikipedia.name

'search_wikipedia'

In [15]:
# 工具的描述
search_wikipedia.description

'search_wikipedia(query: str) -> str - 打开维基百科搜索并获得页面的摘要'

In [16]:
# 将工具格式化为 OpenAI 函数
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'search_wikipedia(query: str) -> str - 打开维基百科搜索并获得页面的摘要',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [17]:
# 调用
search_wikipedia({"query": "langchain"})

"页面: LangChain\n摘要: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n页面: DataStax\n摘要: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\n页面: Sentence embedding\n摘要: In natural language processing, a sentence embedding refers to a numeric representation of a sentence in the form of a vector of real numbers which encodes meaningful semanti

## 2.5-通过API定义function案例

In [18]:
# openapi_spec_to_openai_fn可以把json格式的API定义转换成openai的function call格式
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
# OpenAPISpec是标准化的API格式定义
from langchain.utilities.openapi import OpenAPISpec

In [19]:
# json格式的API定义
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [ ]:
# 从text中导入API的详细定义
spec = OpenAPISpec.from_text(text)

In [21]:
# 转换成openai的fuction call格式
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [22]:
# 查看fuction的定义
pet_openai_functions

[{'name': 'listPets',
  'description': 'List all pets',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'limit': {'type': 'integer',
       'maximum': 100.0,
       'schema_format': 'int32',
       'description': 'How many items to return at one time (max 100)'}},
     'required': []}}}},
 {'name': 'createPets',
  'description': 'Create a pet',
  'parameters': {'type': 'object', 'properties': {}}},
 {'name': 'showPetById',
  'description': 'Info for a specific pet',
  'parameters': {'type': 'object',
   'properties': {'path_params': {'type': 'object',
     'properties': {'petId': {'type': 'string',
       'description': 'The id of the pet to retrieve'}},
     'required': ['petId']}}}}]

In [27]:
# 导入模型
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

In [28]:
# 设置模型温度系数并传入function
model = ChatOpenAI(temperature=0,openai_api_key="your_api_key").bind(functions=pet_openai_functions)

In [29]:
# 输入query，查看模型调用的function以及返回信息
model.invoke("这三只宠物的名字叫什么？")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"params":{"limit":3}}', 'name': 'listPets'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 133, 'total_tokens': 149}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-1231ac8c-cb93-4cd3-86df-90887d8f1427-0')

In [30]:
model.invoke("告诉我id为42的宠物的消息")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"path_params":{"petId":"42"}}', 'name': 'showPetById'}}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 131, 'total_tokens': 150}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-043dd83d-7018-4f6d-9942-e82e97413cb2-0')

# 3、 路由

展示一个函数调用的例子，用于在两个候选函数之间做出决策。



## 3.1-Tool转换为Function

鉴于我们上面提到的工具，让我们将它们格式化为 OpenAI 函数，并展示相同的行为。

In [32]:
# 将工具格式化为 OpenAI 函数
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0,openai_api_key="your_api_key").bind(functions=functions)

In [33]:
# 模型调用
model.invoke("圣佛朗西斯科现在的温度是多少？")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 167, 'total_tokens': 192}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-8c00aa0d-fdb3-41b3-a810-774517009192-0')

In [34]:
# 模型调用
model.invoke("什么是langchain？")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Langchain"}', 'name': 'search_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 154, 'total_tokens': 170}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-7d05c975-1106-46a7-84f1-a6379206ba16-0')

In [35]:
# 使用template构造prompt
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是个乐于助人的助理"),
    ("user", "{input}"),
])

# 创建处理链，将 prompt和model连接起来
chain = prompt | model

In [36]:
# 输入query进行调用
chain.invoke({"input": "圣佛朗西斯科现在的温度是多少？"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 181, 'total_tokens': 206}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-050edbe4-99f1-43d8-a381-fbaf60b0b43b-0')

In [37]:
# 导入输出解析的包
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [38]:
# 创建处理链，将 prompt、model 和 OpenAIFunctionsAgentOutputParser 连接起来
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [39]:
# 调用
result = chain.invoke({"input": "圣佛朗西斯科现在的温度是多少？"})

In [40]:
# 打印返回的类型，可以判断是否产生function的调用
type(result)

langchain_core.agents.AgentActionMessageLog

In [41]:
# 查看调用的tool
result.tool

'get_current_temperature'

In [42]:
# 查看tool的输入，result.message_log可以查看调用结果
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [45]:
# 调用的获取温度的工具
get_current_temperature(result.tool_input)

/var/folders/6f/wm_jrpx102704v2s3pdk8lzm0000gn/T/ipykernel_12575/2418200860.py:39: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'当前的温度是 15.8°C'

In [46]:
# 继续调用
result = chain.invoke({"input": "你好!"})

In [47]:
# 打印返回的类型，可以判断是否产生function的调用
type(result)

langchain_core.agents.AgentFinish

In [48]:
# 查看返回值
result.return_values

{'output': '你好!有什么可以帮助你的吗？'}

## 3.2-通过route进行tools的选择

In [49]:
"""
route会根据result进行tools的选择：
AgentFinish：表示已经完成，可以输出
AgentActionMessageLog：表示未完成，需要继续进行route调用tools
"""
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [50]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [51]:
chain.invoke({"input": "你好！"})

'你好！有什么可以帮助你的吗？'

In [52]:
result = chain.invoke({"input": "圣弗朗西斯科的天气现在怎么样？"})
result

/var/folders/6f/wm_jrpx102704v2s3pdk8lzm0000gn/T/ipykernel_12575/2418200860.py:39: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'当前的温度是 15.8°C'

In [53]:
result = chain.invoke({"input": "什么是langchain?"})
result

"页面: LangChain\n摘要: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n页面: DataStax\n摘要: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\n页面: Sentence embedding\n摘要: In natural language processing, a sentence embedding refers to a numeric representation of a sentence in the form of a vector of real numbers which encodes meaningful semanti

# 四、英文版提示

我们总结一下完整的调用流程：

构造Prompt --> 调用模型 --> 解析模型返回的结果 --> 进行路由选择对应的tool


In [54]:
@tool
def search(query: str) -> str:
    """"Search for weather online"""
    return "42f"

In [55]:
# 搜索工具的函数名
print(search.name)
#搜索工具的功能描述（即函数注释）
print(search.description)
# 搜索工具需要传递的参数
print(search.args)

search
search(query: str) -> str - "Search for weather online
{'query': {'title': 'Query', 'type': 'string'}}


In [56]:
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

In [57]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [58]:
search.run("sf")

'42f'

In [60]:
# 导入所需的库
import requests
from pydantic import BaseModel, Field
import datetime

# 定义输入类（input schema）
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for") #要获取天气数据的位置的纬度
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for") #要获取天气数据的位置的经度

# 使用 @tool 装饰器并指定输入模型
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """"Fetch current temperature for given coordinates."""
    
    # Open Meteo API 的URL
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # 请求参数
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # 发送 API 请求
    response = requests.get(BASE_URL, params=params)
    
    # 检查响应状态码
    if response.status_code == 200:
        # 解析 JSON 响应
        results = response.json()
    else:
        # 处理请求失败的情况
        raise Exception(f"API Request failed with status code: {response.status_code}")

    # 获取当前 UTC 时间
    current_utc_time = datetime.datetime.utcnow()
    
    # 将时间字符串转换为 datetime 对象
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    
    # 获取温度列表
    temperature_list = results['hourly']['temperature_2m']
    
    # 找到最接近当前时间的索引
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    
    # 获取当前温度
    current_temperature = temperature_list[closest_time_index]
    
    # 返回当前温度的字符串形式
    return f'The current temperature is {current_temperature}°C'


In [61]:
# 工具的名字
print(get_current_temperature.name)
# 工具的功能描述
print(get_current_temperature.description)
# 工具的输入参数
print(get_current_temperature.args)

get_current_temperature
get_current_temperature(latitude: float, longitude: float) -> dict - "Fetch current temperature for given coordinates.
{'latitude': {'title': 'Latitude', 'description': 'Latitude of the location to fetch weather data for', 'type': 'number'}, 'longitude': {'title': 'Longitude', 'description': 'Longitude of the location to fetch weather data for', 'type': 'number'}}


In [62]:
# 导入openai的模板
from langchain.tools.render import format_tool_to_openai_function

format_tool_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - "Fetch current temperature for given coordinates.',
 'parameters': {'type': 'object',
  'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [63]:
get_current_temperature({"latitude": 13, "longitude": 14})

/var/folders/6f/wm_jrpx102704v2s3pdk8lzm0000gn/T/ipykernel_12575/1889563274.py:39: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'The current temperature is 33.8°C'

In [64]:
import wikipedia

# 定义维基百科搜索的tool
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]: #取前三个页面标题
        try:
            #使用 wikipedia 模块的 page 函数，获取指定标题的维基百科页面对象。
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False) 
            # 获取页面摘要
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [65]:
# 将工具格式化为 OpenAI 函数
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [67]:
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
# 将工具格式化为 OpenAI 函数
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0,openai_api_key="your_api_key").bind(functions=functions)

In [68]:
# 使用template构造prompt
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])

# 创建处理链，将 prompt和model连接起来
chain = prompt | model

In [69]:
# 导入输出解析的包
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [70]:
# 调用
result = chain.invoke({"input": "what is the weather in sf right now"})

/var/folders/6f/wm_jrpx102704v2s3pdk8lzm0000gn/T/ipykernel_12575/1889563274.py:39: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


In [71]:
result

'The current temperature is 15.8°C'

In [72]:
# 调用并查看结果
chain.invoke({"input": "hi!"})

'Well, hello there! How can I assist you today?'

In [73]:
# 调用
result = chain.invoke({"input": "langchain?"})
result

'Hmm, I\'m not sure what "langchain" refers to. Could you provide more context or clarify your question?'

In [74]:
# 调用
result = chain.invoke({"input": "What is the weather in san francisco right now?"})
result

/var/folders/6f/wm_jrpx102704v2s3pdk8lzm0000gn/T/ipykernel_12575/1889563274.py:39: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'The current temperature is 15.8°C'

In [75]:
# 导入tool包
from langchain.agents import tool

In [76]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [77]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [78]:
# 调用并查看结果
chain.invoke({"input": "hi!"})

'Well, hello there! How can I assist you today?'

In [79]:
# 调用
result = chain.invoke({"input": "What is langchain?"})
result

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: DataStax\nSummary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\nPage: Sentence embedding\nSummary: In natural language processing, a sentence embedding refers to a numeric representation of a sentence in the form of a vector of real numbers which encod

In [80]:
# 调用
result = chain.invoke({"input": "What is the weather in san francisco right now?"})
result

/var/folders/6f/wm_jrpx102704v2s3pdk8lzm0000gn/T/ipykernel_12575/1889563274.py:39: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'The current temperature is 15.8°C'